In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import pandas as pd

In [ ]:
## the model input neighbour list, idx, probability, time, the previous infections, parameters, population size 
## the model output for kth ensemble, the county everytime
## it iterates every time
NewInf(:,:,k)=branching(nl,part,w,t,NewInf(:,:,k),para(:,k),population);

In [3]:
WN = np.loadtxt('W_avg.csv')